In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.datasets
from sklearn.ensemble import RandomForestRegressor as rf, GradientBoostingRegressor as gbr, BaggingRegressor as br
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor as knr
from xgboost import XGBRegressor
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn import metrics
pd.set_option("display.max.columns", None)
# pd.set_option("display.max.rows", None)

In [2]:
#storing the data from a file path to the varriable
house_price_train_dataset_path = '/kaggle/input/hse-m-psmsimmo-p-347795-1-apartmentprice/train.csv'
house_price_train_dataset_table = pd.read_csv(house_price_train_dataset_path)
house_price_test_dataset_path = '/kaggle/input/hse-m-psmsimmo-p-347795-1-apartmentprice/test.csv'
house_price_test_dataset_table = pd.read_csv(house_price_test_dataset_path)
(house_price_test_dataset_table. index)

# check shape of my data
house_price_train_dataset_table.shape

# out put number of empty cells, grouping by columns
# house_price_train_dataset_table.isnull().sum()


(40508, 12)

In [3]:
# deleting deletable rows with empty cells if they are not much
house_price_train_dataset_table.dropna(subset=['District', 'Age','Walls'], inplace=True)
house_price_train_dataset_table.isnull().sum()

#Replacing  undeletable rows with empty cells with unknown for those cells
house_price_train_dataset_table = house_price_train_dataset_table.fillna("Unknown")
house_price_test_dataset_table = house_price_test_dataset_table.fillna("Unknown")

# house_price_train_dataset_table.isnull().sum()
# house_price_train_dataset_table.shape
# print(house_price_train_dataset_table['Age'].unique())

In [4]:
# print(x_test['Age'].unique())
# x_train.head()
# house_price_train_dataset_table.shape
# house_price_train_dataset_table.isnull().sum()[0:60]
# x_train.dtypes[120:145]

In [5]:
house_price_test_dataset_table.head()
# house_price_train_dataset_table.columns.difference(house_price_test_dataset_table.columns).tolist()

,Id,Room,Size,Balcony,Floor,FloorsTotal,Walls,Age,Lift,District,Okrug
0,40509,2,58,Unknown,2,5,Unknown,вторичка,Unknown,Богородский,ВАО
1,40510,5,266,лоджия,3,14,Unknown,вторичка,грузовой,Мещанский,ЦАО
2,40511,5,197,лоджия,5,9,Unknown,вторичка,грузовой,Басманный,ЦАО
3,40512,5,233,лоджия,3,14,Unknown,вторичка,грузовой,Мещанский,ЦАО
4,40513,2,69,Unknown,2,6,Unknown,вторичка,пассажирский,Красносельский,ЦАО


In [6]:
# replacing categorical features for test and train data of 2 and 3 values with 0 and 1 or 1, 2, 3
house_price_train_dataset_table['Age'] = house_price_train_dataset_table['Age'].replace({'новостройка': 1, 'вторичка': 0, 'Unknown': 0.5})
house_price_test_dataset_table['Age'] = house_price_test_dataset_table['Age'].replace({'новостройка': 1, 'вторичка': 0, 'Unknown': 0.5})
house_price_train_dataset_table['Lift'] = house_price_train_dataset_table['Lift'].replace({'грузовой': 3, 'пассажирский': 2, 'Unknown': 1})
house_price_test_dataset_table['Lift'] = house_price_test_dataset_table['Lift'].replace({'грузовой': 3, 'пассажирский': 2, 'Unknown': 1})
house_price_train_dataset_table['Balcony'] = house_price_train_dataset_table['Balcony'].replace({'лоджия': 3, 'балкон': 2, 'Unknown': 1})
house_price_test_dataset_table['Balcony'] = house_price_test_dataset_table['Balcony'].replace({'лоджия': 3, 'балкон': 2, 'Unknown': 1})
# replacing categorical features for test and train data with more than 3 values with dumies of 000 010 001 etc
house_price_train_dataset_table = pd.get_dummies(house_price_train_dataset_table, columns=['Walls','Okrug', 'District'])
house_price_test_dataset_table = pd.get_dummies(house_price_test_dataset_table, columns=['Walls','Okrug', 'District'])



# Creating missing columns in test and train data
house_price_train_dataset_table.insert(loc=23, column='District_Unknown',value=0)
house_price_train_dataset_table.insert(loc=13, column='Okrug_Unknown',value=0)
house_price_train_dataset_table.insert(loc=7, column='Walls_Unknown',value=0)
house_price_test_dataset_table.insert(loc=37, column='District_Внуково',value=0)

# house_price_train_dataset_table.shape
# house_price_test_dataset_table.columns.difference(house_price_train_dataset_table.columns).tolist()
# house_price_train_dataset_table = house_price_train_dataset_table.drop(['Balcony_балкон', 'Walls_кирпичный', 'Age', 'Lift', 'Okrug_САО', 'Okrug_СЗАО', 'Okrug_Unknown'], axis=1)
# house_price_test_dataset_table =  house_price_test_dataset_table.drop(['Balcony_балкон', 'Walls_кирпичный', 'Age', 'Lift', 'Okrug_САО', 'Okrug_СЗАО', 'Okrug_Unknown'], axis=1)
# Balcony_балкон, Walls_кирпичный, Age, Lift, Okrug_САО, Okrug_СЗАО, Okrug_Unknown

In [7]:
# x_test['Age'] = house_price_test_dataset_table['Age'].replace({'Unknown': 0.5})

In [8]:
#Deleting ID before fedding it to the model
house_price_train_dataset_table = house_price_train_dataset_table.drop(['Id'], axis=1)
house_price_test_dataset_table =  house_price_test_dataset_table.drop(['Id'], axis=1)

In [9]:
#undferstanding the correlation between the various features in the data(positive or negative)
# correlation=house_price_train_dataset_table.corr()
# plt.figure(figsize=(20,20))
# sns.heatmap(correlation, cbar=True, square=True, fmt='.1f', annot=True, annot_kws={'size':5}, cmap='Blues')

corr = house_price_train_dataset_table.corr()
print (corr['Price'].sort_values(ascending=False)[120:180])


District_Вешняки                  -0.027999
District_Южное Чертаново          -0.028794
District_Дмитровский              -0.029103
District_Люблино                  -0.029625
District_Южное Орехово Борисово   -0.030070
District_Митино                   -0.031793
District_Отрадное                 -0.032526
District_Гольяново                -0.032924
District_Бирюлёво Восточное       -0.033900
District_Марьино                  -0.034255
Okrug_СЗАО                        -0.039250
District_Южное Бутово             -0.043770
District_Некрасовка               -0.043882
Okrug_САО                         -0.044069
Okrug_ЮЗАО                        -0.053876
Walls_блочный                     -0.074960
Okrug_СВАО                        -0.077004
Okrug_ЮАО                         -0.084274
Okrug_ВАО                         -0.086298
Okrug_ЮВАО                        -0.090859
Balcony                           -0.132296
Walls_панельный дом               -0.211769
Walls_Unknown                   

In [10]:
# spliting the data
x_train=house_price_train_dataset_table.drop(['Price'], axis=1)
y_train=house_price_train_dataset_table['Price']
x_test=house_price_test_dataset_table
# X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=2)
# print(X.shape, X_train.shape, X_test.shape )

In [11]:
# training the model
# model = knr()
# model = br()
# model = gbr(learning_rate=0.1, random_state=60)
# model = SVR()
model=rf(n_estimators = 1000, random_state = 42) #BEST
# model=XGBRegressor()
model.fit(x_train, y_train)

RandomForestRegressor(n_estimators=1000, random_state=42)

In [12]:
# predicting the data
# Reorder the test dataframe to match the order of the train dataframe
x_test = x_test.reindex(columns = x_train.columns)
test_data_prediction=model.predict(x_test)
# checking errors with R Sqaured and mean absolute error
# score_1=metrics.r2_score(Y_train, training_data_prediction)
# score_2=metrics.mean_absolute_error(Y_train, training_data_prediction)
# print('R square error: ', score_1)
# print('mean absolute error: ', score_2)
test_data_prediction=pd.DataFrame(test_data_prediction)
pd.options.display.float_format = '{:.8f}'.format
# test_data_prediction.insert(loc=0, column='Id', value=range(0,len(test_data_prediction)))
# test_data_prediction.set_index('Id')
test_data_prediction.index.name = 'Id'
test_data_prediction.index = test_data_prediction.index + 40509
test_data_prediction.rename(columns = {0:'Price'}, inplace = True)
# pd.set_option('display.max_rows', None)
test_data_prediction.to_csv(f"Submission_file_oct-2021-101.csv")
print(test_data_prediction)




                   Price
Id                      
40509   8942865.00000000
40510 248216440.00000000
40511 106741930.00000000
40512 159642740.00000000
40513  27479155.05952381
...                  ...
50631  12264180.00000000
50632  12583901.66666667
50633   5424660.00000000
50634  10608222.00000000
50635   8738310.00000000

[10127 rows x 1 columns]


In [13]:
# #a view of the train data set
# print(house_price_train_dataset_table)

In [14]:
# #a view of the first 5 rows of the train dataset
# house_price_train_dataset_table.head()

In [15]:
# #a view of the test data set
# print(house_price_test_dataset_table)

In [16]:
# #a view of the first 5 rows of the test dataset
# house_price_test_dataset_table.head()

In [17]:
# #number of attribute cells without inputs
# house_price_train_dataset_table.isnull().sum()

In [18]:
#code to view the different attributes of each feature
# house_price_train_dataset_table['Wall'].value_counts()

In [19]:
# code to select particular columns of a pandas table
# q=house_price_train_dataset_table.iloc[:, [2,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99]]
# print(q)

In [20]:
house_price_test_dataset_table.isnull().sum()

Room                        0
Size                        0
Balcony                     0
Floor                       0
FloorsTotal                 0
                           ..
District_Южное Чертаново    0
District_Южнопортовый       0
District_Якиманка           0
District_Ярославский        0
District_Ясенево            0
Length: 144, dtype: int64

In [21]:
house_price_train_dataset_table.isnull().sum()

Room                        0
Price                       0
Size                        0
Balcony                     0
Floor                       0
                           ..
District_Южное Чертаново    0
District_Южнопортовый       0
District_Якиманка           0
District_Ярославский        0
District_Ясенево            0
Length: 145, dtype: int64